# Sizing Assumptions

## Airloop

In [58]:
# assumptions for nominal supply airflow
m = 33
dT = 15
c = 1005
ro = 1.17
V = m/ro
print(V)

V_cfm = V/0.00047194745
print(V_cfm)

28.205128205128208
59763.28128296531


In [59]:
V*200/0.7/V_cfm

0.13484212857142858

## Chilled water flow

In [44]:
Cn = 471000 #W
Cn_ton = Cn/3500
print(Cn_ton)

V_gpm = 1.2 * Cn_ton
V = V_gpm * 0.0000630901964
ro_w = 1000
m_chw_flow_nominal = V * ro_w
print(m_chw_flow_nominal, "; In our model we assume", 12, "kg/s")

134.57142857142858
10.188165430080002 ; In our model we assume 12 kg/s


### Comparison with literature

[source](https://www.trane.com/content/dam/Trane/Commercial/global/products-systems/education-training/engineers-newsletters/waterside-design/ADM-APN051-EN.pdf)

In [45]:
m_chw_flow_nominal = 12 #kg/s
m_chw_flow_nominal_gpm = (12/1000)/0.0000630901964
print("Our model nominal evaporator flow in gpm/ton is:", m_chw_flow_nominal_gpm/Cn_ton)
print("Source provides for a 130 ton chiller a nominal evaporator flow in gpm/ton of", 1.5)

Our model nominal evaporator flow in gpm/ton is: 1.4134046113429595
Source provides for a 130 ton chiller a nominal evaporator flow in gpm/ton of 1.5


### Condenser water flow check

In [50]:
# 3 gpm is 0.14 kg/s, see Paul's method based on ASHRAE Standard 550 / 590
0.14*130, "Our nominal value is about", 20, "kg/s"

(18.200000000000003, 'Our nominal value is about', 20, 'kg/s')

## Condenser, Evaporator, Coil, and WSE Pressure Drops
[based on examples towards the end of this taylor engineering article](https://www.taylor-engineering.com/wp-content/uploads/2020/04/ASHRAE_Journal_-_How_to_Design_Control_Waterside_Economizers.pdf), it can be assumed that the condenser water side pressure drop equals the pressure drop on the evaporator side. Same goes for WSE HX.

### Chiller (based on a 150 ton Carrier Screw Chiller)
[source](https://chillerscarrier.com/assets/30xw-9pd.pdf)

In [53]:
evap_dp = 33.2 #kPa
evap_flow = 17.91 #kg/s
m_chw_flow_nominal = 12 #kg/s

cond_dp = 42.1 #kPa
cond_flow = 22.39 #kg/s
m_cw_flow_nominal = 39.7/2 #kg/s

# adjusted to our nominal flow

evap_dp_adjusted = evap_dp * (m_chw_flow_nominal/evap_flow)**2
print("Adjusted evaporator pressure drop:", evap_dp_adjusted, " kPa")
cond_dp_adjusted = cond_dp * (m_cw_flow_nominal/cond_flow)**2
print("Adjusted condenser pressure drop:", cond_dp_adjusted, " kPa")

Adjusted evaporator pressure drop: 18.957830688210848  kPa
Adjusted condenser pressure drop: 33.089860089154044  kPa


### Cooling tower

[source](https://www.baltimoreaircoil.com/download_api_endpoint/1178/j120_tr_pumpingpiping_20150914.pdf)

In [52]:
ro_w = 1000 # kJ/(kg*K)
gr = 9.81 #kg/(s2)
height = 1.5 #m, 5ft - confirmed per communication with Paul

coo_tow_dp = ro_w * gr * height
print("Tower dp is: ",coo_tow_dp/1000, "kPa")

Tower dp is:  14.715 kPa


### Cooling coil

#### Airside

**Sources used**

[has a downloadable excel tool](https://www.engineeringtoolbox.com/duct-friction-pressure-loss-d_444.html)

[examples by stulz.de](https://www.stulz.de/de/newsroom/blog/aer-eine-neue-kennzahl-fuer-die-effizienz-von-luftbewegung-in-rechenzentren-352/), 
using specific fan power slightly above example 1 in the assumptions below, scaled for our nominal flow as above.

Ducts: short duct ((veloc^2 x ro)/2) + plenum : 100 Pa

**Resulting reasonable assumptions**

coil 200 Pa , also corresponds well with this [source](https://www.trane.com/content/dam/Trane/Commercial/global/products-systems/education-training/engineers-newsletters/standards-codes/ADM-APN070-EN_06032019.pdf)

note: plenum almost negligible

#### Waterside

Scaled from [source, table 1](https://www.trane.com/content/dam/Trane/Commercial/global/products-systems/education-training/engineers-newsletters/standards-codes/ADM-APN070-EN_06032019.pdf

In [55]:
m_chw = 12

# assuming the pressure drop scales with the square of the velocity and the pipe diameter stays the same:
dp = 33178.6 * (12/40)**2
dp

2986.0739999999996

## Cooling Tower

Approach temperature [Source (PG&E, leaving tower water temperature minus ambient wet bulb
temperature](https://www.pge.com/includes/docs/pdfs/mybusiness/energysavingsrebates/incentivesbyindustry/DataCenters_BestPractices.pdf) recommends 5 - 7 F, 
conservatively assuming

In [47]:
approachT = 4 #K

### Sizing the tower fan

In [63]:
# cold water temperature (entering the condenser)
T_cond_in_max = 297 #K
T_wet_bulb_max = 296 #K

T_cond_in_max_C = 297 - 273 #K
T_wet_bulb_max_C = 296 - 273 #K

print("T_cond_in_max_C:", T_cond_in_max_C)
print("T_wet_bulb_max_C:", T_wet_bulb_max_C)

T_cond_in_max_C: 24
T_wet_bulb_max_C: 23


## Pauls method based on ASHRAE Standard 550 / 590

In [64]:
# ASHRAE Standard 550 / 590, [source](https://www.trane.com/content/dam/Trane/Commercial/global/products-systems/education-training/engineers-newsletters/waterside-design/adm-apn045-en_0912.pdf) 
# Tower is sized at 3.0 GPM per ton of plant capacity.  Note that some designs reduce this to 2.0 or 2.5 GPM per ton. 
# Chiller efficiency is .57 KW / Ton.  Some machines are even more efficient - but it is hard to get much better 
# than .5 KW / ton at design conditions.
# Tower fan efficiency .046 KW / Ton
# Condenser water pump .054 KW / Ton
# Tower + Pump = .1 KW / Ton

# In terms of overall plant energy use it is valuable to think of the overall plant running around .6 - .8 KW / Ton.  
# Of that amount 85% is attributable to the chiller and roughly 15% to the tower fan and pump.  
# So in terms of the overall lift done by the plant 
# (difference between the CHW temperature and the outdoor air wet bulb) - the tower is roughly 
# five times more efficient than the chiller.  This is due mostly to the fact that the tower is evaporating water - rather 
# than trying to mechanically cool it.  So the more work we can have the tower due and the less the chiller needs to do the better.  
# This is why strategies like the use of WSE and CDW are so effective in reducing total plant efficiency.

In [66]:
# Tower fan efficiency .046 KW / Ton
print("Tower nominal fan power is thus: ", .046 * 142, "kW")

Tower nominal fan power is thus:  6.532 kW


## Piping

[source](https://www.advantageengineering.com/fyi/166/advantageFYI166.php)

6" CW at 2200 Pa/100ft

5" CHW Pa/100ft

For those with access, the compiled spreadsheet is [here](https://docs.google.com/spreadsheets/d/1XtaqeURHSJD2tsk9S-GBtri8GY4wZl62JM6LV3KOiVo/edit?usp=sharing)
For the others, please see the ChillerPlant package models. The piping resistances are for the most part passed as pressure drops in valves 